use firefox plugin, select all songs, then right click, copy selected links

In [90]:
# use firefox F12, copy the POST request, and then copy as cURL
# the use https://curl.trillworks.com/, convert cURL to python codes
import re
import os
import json
import requests
import logging

logging.basicConfig(level=logging.DEBUG)



def get_song_info(song_id):
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:67.0) Gecko/20100101 Firefox/67.0',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'zh-CN,en;q=0.7,en-US;q=0.3',
        'Referer': 'http://music.wandhi.com/?url=https%3A%2F%2Fmusic.163.com%2Fsong%3Fid%3D'+song_id,
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'X-Requested-With': 'XMLHttpRequest',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Pragma': 'no-cache',
        'Cache-Control': 'no-cache',
    }

    data = {
      'input': 'https://music.163.com/song?id='+song_id,
      'filter': 'url',
      'type': '_',
      'page': '1'
    }

    r = requests.post('http://music.wandhi.com/', headers=headers, data=data)

    if 200 != r.status_code:
        msg = "Failed to get song info, song id %r, status is %r" % (song_id, r.status_code)
        logging.debug(msg)
        return None

    song_info = json.loads(r.content)
    song_data = song_info.get('data', [])
    if song_data and list == type(song_data):
        song_data = song_data[0]
    else:
        msg = "Failed to parse song info: {}".format(song_data)
        logging.debug(msg)
        return None

    return song_data
# song_url, song_author, song_title

In [91]:
def download_file(url, ):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    # f.flush()
    return local_filename

In [92]:
import os

def download_song(song_url, song_title):
    filename = download_file(song_url)
    file_ext = os.path.splitext(filename)[-1]
    dst_file = song_title+file_ext
    os.rename(filename, dst_file)
    return dst_file

In [93]:
"""
target play list
# https://music.163.com/song?id=34619045
# all songs links
"""
all_songs = \
"""
https://music.163.com/song?id=34619045
https://music.163.com/artist?id=38883
https://music.163.com/album?id=3290158
https://music.163.com/song?id=34619046
https://music.163.com/song?id=34619047
https://music.163.com/song?id=34619049
https://music.163.com/song?id=34619050
https://music.163.com/song?id=34619051
https://music.163.com/song?id=34619052
https://music.163.com/song?id=34619053
https://music.163.com/song?id=34619054
https://music.163.com/song?id=34619055
https://music.163.com/song?id=34619056
https://music.163.com/song?id=34619057
https://music.163.com/song?id=34619058
https://music.163.com/song?id=34619059
https://music.163.com/song?id=34619060
https://music.163.com/song?id=34619061
https://music.163.com/song?id=34619062
https://music.163.com/song?id=34619063
https://music.163.com/song?id=34619064
https://music.163.com/song?id=34619065
https://music.163.com/song?id=34619066
https://music.163.com/song?id=34619067
https://music.163.com/song?id=34619068
https://music.163.com/song?id=34619069
https://music.163.com/song?id=34619070
https://music.163.com/song?id=34619071
https://music.163.com/song?id=34619072
https://music.163.com/song?id=34619073
https://music.163.com/song?id=34619074
https://music.163.com/song?id=34619075
https://music.163.com/song?id=34619048
https://music.163.com/song?id=34509684
https://music.163.com/artist?id=12023288
https://music.163.com/artist?id=15166305
https://music.163.com/album?id=3279569
https://music.163.com/song?id=34509685
https://music.163.com/artist?id=190807
https://music.163.com/song?id=34509686
https://music.163.com/artist?id=1146083
https://music.163.com/artist?id=1146085
https://music.163.com/artist?id=29405254
https://music.163.com/song?id=34509687
https://music.163.com/song?id=34509688
https://music.163.com/song?id=34509689
https://music.163.com/song?id=34509690
"""

In [94]:
def download_song_by_id(song_id):
    song_file = ''
    song_info = get_song_info(song_id)
    logging.debug(song_info)
    if not song_info:
        logging.error(song_id, " get null song_info")
        return ''
    logging.debug("song_info: {}".format(song_info))
#     song_url = song_info.get('url', '')
#     song_title = song_info.get('title', '')
#     if not song_url or song_title:
#         logging.error(song_id, " get null song_title or url")
#         return ''
#     song_file = download_song(song_url, song_title)
#     logging.info(song_file)
    return song_file

In [98]:
song_id_list = []
for song_url in all_songs.splitlines():
    song_id = re.sub('^.*id=', '', song_url)
    song_id = song_id.strip()
    if song_id:
        song_id_list.append(song_id)

logging.debug(song_id_list)
assert song_id_list, "song id list is empty"

from multiprocessing.dummy import Pool
pool = Pool(processes=len(song_id_list))
song_files = pool.map(download_song_by_id, song_id_list)
pool.close()
pool.join()

# logging.info(song_files)

# filename = download_song_by_id('34619046')
# print(filename)

print("xxxxxxxxxxxxxxx")
os.listdir()

''